![](https://i.imgur.com/WEqQ4Cn.png)

# Índice
* [Introducción](#introduccion)
* [Explicación de Variables](#explicacion-variables)
* [Visualización de Datos](#visualización-datos)
    - [Conclusión](#conclusion-datos)
    - [Análisis de Datos](#analisis-datos)
* [Selección de Variables](#seleccion-variables)
* [Normalización de los Datos](#normalizacion-datos) 
* [Clasificadores Vanilla](#clasificadores-vanilla)
    - [Resultados](#resultados-vanilla)
        + [Definición de Métricas](#metricas)
* [Aplicando GridSearch + Cross Validation](#grid-search)
    - [Clasificadores](#clasificadores-grid)
    - [Resultados](#resultados-grid)
* [Aplicando PCA + GridSearch + Cross Validation](#pca)
    - [Implementando PCA de Scikit-Learn](#implementar-pca)
    - [Clasificadores](#clasificadores-pca)
    - [Resultados](#resultados-pca)
* [Conclusión](#conclusion)
    - [Tabla de Métricas](#tabla-metricas)
* [Bibliografía](#bibliografia)

<a id="introduccion"></a>
# Introducción

## Contexto

El presente dataset trata sobre un videojuego del género MOBA (multijugador de arena de batalla en línea) denominado **"League of Legends"**, abreviado **"LoL"**, en donde se enfrentan **2 equipos**, uno azul y otro rojo, en un mapa que contiene **3 carriles y una jungla**, que se encuentra entre los mismos. Cada equipo posee **5 integrantes** y cada uno ocupa un rol particular dentro del mismo, siendo el objetivo el destruir el **"nexo"**, un edificio crítico, del otro equipo para ganar la partida.

## Dataset

Específicamente, este dataset contiene diferentes estadísticas de los **primeros 10 minutos** de juego de alrededor **10 mil partidas** de jugadores con **"High Elo"**, es decir, jugadores que alcanzaron un nivel alto dentro de la clasificación del juego. 

## Objetivo

La columna **'blueWins'** es el valor objetivo (el valor que estamos tratando de predecir). Un valor de 1 significa que el equipo azul ganó partida, por otro lado, un valor de 0 indica que el equipo azul perdió.

In [ ]:
# Importamos algunas librerías que vamos a utilizar a lo largo de este trabajo.
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Importamos el dataset previamente descrito.
path = "../input/league-of-legends-diamond-ranked-games-10-min"

df = pd.read_csv(path + "/high_diamond_ranked_10min.csv")
df.head()

Como no nos va a servir para el análisis de los datos, eliminamos la siguiente columna:
* **'gameId':** identificador único utilizado en la API del juego para acceder a los datos específicos de una partida.

In [ ]:
# Eliminamos la columna 'gameId'
df = df.drop('gameId',  axis=1)

Una vez quitadas las columnas redundantes e innecesarias, una buena práctica es generar una matriz de correlación. Esta matriz nos permitirá conocer el grado de correlación que existe entre las diferentes variables que pueblan el dataset. Pero...

### Qué significa esto? 

La matriz de correlación muestra los **valores de correlación de Pearson**, que miden el grado de relación lineal entre cada par de elementos o variables. Los valores de correlación se pueden ubicar entre -1 y +1. Sin embargo, en la práctica, los elementos por lo general tienen correlaciones positivas. Si los dos elementos tienden a aumentar o disminuir al mismo tiempo, el valor de correlación es positivo.

#### Interpretación

**Se utiliza la matriz de correlación para evaluar la fuerza y dirección de la relación entre dos variables.** Un valor de correlación alto y positivo indica que los elementos miden la misma destreza o característica. Si los elementos no están altamente correlacionados, entonces los elementos pudieran medir diferentes características o no estar claramente definidos.

### Aplicado a nuestro caso

Por lo tanto, aplicaremos la matriz de correlación para observar que variables del dataset son candidatas a salir del modelo, ya que explican o miden las mismas características. Esto lo realizaremos con el siguiente código:

In [ ]:
plt.figure(figsize=(20,15))
sns.heatmap(round(df.corr(),1), cmap="coolwarm", annot=True, linewidths=.5)
plt.show()

### Conclusión de Matriz de Correlación

Como podemos observar en la matriz, existen variables que están altamente correlacionadas, es decir, explican las mismas cosas. Por lo tanto, no ayudan a clasificar mejor, si no que muestran los mismos datos que otra columna. Esto ocurre, por ejemplo, con las columnas:
* **"RedKills":** cantidad de asesinatos del equipo rojo.
* **"BlueDeaths":** cantidad de muertes del equipo azul.
La cantidad de asesinatos del equipo rojo son la cantidad de muertes que tiene el equipo azul. Por lo tanto, lo acertado seria eliminar una columna, ya que una explica la otra.
> _**Aclaración:** dentro del juego, un jugador puede morir sin haber sido asesinado por un miembro del otro equipo, es decir, no sumaría a la cantidad de kills, como por ejemplo, morir a causa de un **"monstruo neutral"**. Sin embargo, estamos hablando de partidas de **"high elo"**, en otras palabras, partidas con jugadores poseen un nivel muy alto de juego, por lo que es poco probable que ocurra. Cosas como estas deberian ser tomadas en cuenta para la selección de variables._

<a id="explicacion-variables"></a>
# Explicación de Variables

## Principales variables dentro del Juego

### Experiencia

Dentro de **League of Legends** los jugadores utilizan a personajes, denominados **"campeones"**, los cuales poseen un nivel y determinadas habilidades y estadísticas. Un personaje puede subir de nivel obteniendo experiencia, y hacerlo le permite mejorar sus habilidades y stats, lo que le otorga una ventaja frente al enemigo.

### Oro

Por otro lado, tenemos el oro, que es único en la partida y se usa para comprar objetos. Estos últimos permiten a los personajes incrementar aún más sus estadísticas, así como obtener habilidades especiales.

## Como obtenerlos?

Se obtiene experiencia y oro asesinando o destruyendo diferentes objetos dentro del juego, como son:
* **Creeps:** NPCs que pertenecen a ambos equipos. Dan oro cuando los jugadores los matan.
* **Mostruos de Elite:** Monstruos con alto HP (Health Points) y daño que otorgan una bonificación masiva de oro, XP y estadísticas cuando son asesinados por un equipo, estos son:
    * Dragones: Monstruo de élite que otorga bonificaciones al equipo cuando es asesinado. El 4to dragón asesinado por un equipo otorga una bonificación de estadísticas masivas. El 5to dragón (Elder Dragon) ofrece una gran ventaja para el equipo.
    * Heraldos: Monstruo de élite que otorga bonificación de estadísticas cuando es asesinado por un jugador. Ayuda a empujar un carril y destruye estructuras.
* **Jungle Creeps:** NPCs que no pertenecen a NINGÚN EQUIPO. Dan oro y experiencia cuando los jugadores los matan.
* **Estructuras enemigas:** Se deben destruir para llegar al "Nexo" enemigo. Otorgan oro al ser destruidas. Estas estructuras son:
    * Torres
    * Inhibidores.
* **Campeones enemigos:** personajes del juego utulizados por los miembros del equipo opuesto.
* **Wards:** Elemento que un jugador puede poner en el mapa para revelar el área cercana. Muy útil para el control de mapas y objetivos, ya que la visibilidad se ve afectada por la **“niebla de guerra”.**

<a id="visualización-datos"></a>
# Visualización de Datos

Para ver de una manera más clara que variables contribuyen menos o mas a la victoria del equipo azul, podemos realizar diferentes gráficos y observar como están distribuidos los datos:

In [ ]:
data = df
sns.set(font_scale=1.5)

plt.figure(figsize=(20,20))
sns.set_style("whitegrid")

# Cantidad de kills de cada equipo
plt.subplot(321)
sns.scatterplot(x='blueKills', y='redKills', hue='blueWins', data=data)
plt.title('KILLS totales de cada equipo')
plt.xlabel('Equipo Azul')
plt.ylabel('Equipo Rojo')
plt.grid(True)

# Cantidad de asistencias de cada equipo
plt.subplot(322)
sns.scatterplot(x='blueAssists', y='redAssists', hue='blueWins', data=data)
plt.title('ASISTENCIAS totales de cada equipo')
plt.xlabel('Equipo Azul')
plt.ylabel('Equipo Rojo')
plt.tight_layout(pad=1.5)
plt.grid(True)

# Cantidad total de oro de cada equipo
plt.subplot(323)
sns.scatterplot(x='blueTotalGold', y='redTotalGold', hue='blueWins', data=data)
plt.title('ORO total de de cada equipo')
plt.xlabel('Equipo Azul')
plt.ylabel('Equipo Rojo')
plt.tight_layout(pad=1.5)
plt.grid(True)

# Cantidad total de experiencia de cada equipo
plt.subplot(324)
sns.scatterplot(x='blueTotalExperience', y='redTotalExperience', hue='blueWins', data=data)
plt.title('EXPERIENCIA total de de cada equipo')
plt.xlabel('Equipo Azul')
plt.ylabel('Equipo Rojo')
plt.tight_layout(pad=1.5)
plt.grid(True)

# Cantidad total de Wards colocadas por cada equipo
plt.subplot(325)
sns.scatterplot(x='blueWardsPlaced', y='redWardsPlaced', hue='blueWins', data=data)
plt.title('WARDs totales colocadas de cada equipo')
plt.xlabel('Equipo Azul')
plt.ylabel('Equipo Rojo')
plt.tight_layout(pad=1.5)
plt.grid(True)

# Juntamos la cantidad total de minions por equipo
data['blueMinionsTotales'] = df['blueTotalMinionsKilled'] + df['blueTotalJungleMinionsKilled']
data['redMinionsTotales'] = df['redTotalMinionsKilled'] + df['redTotalJungleMinionsKilled']

# Total de minions asesinados por cada equipo
plt.subplot(326)
sns.scatterplot(x='blueMinionsTotales', y='redMinionsTotales', hue='blueWins', data=data)
plt.title('MINIONs totales asesinados de cada equipo')
plt.xlabel('Equipo Azul')
plt.ylabel('Equipo Rojo')
plt.tight_layout(pad=1.5)
plt.grid(True)

plt.show()

<a id="conclusion-datos"></a>
## Conclusión de visualización de datos

De los gráficos previamente visualizados podemos decir que:
* A mayor cantidad de **kills** y **asistencias totales** se incrementan las posibilidades de ganar.
* Esto también se verifica con la cantidad total de **experiencia** y **oro** por equipo, ya que a mayor cantidad de **kills** y **asistencias totales**, mayor cantidad de los primeros valores. La tendencia dice que mientras más **experiencia** y **oro** tenga un equipo, mayores serán sus posibilidades de ganar.
* Las **wards** totales colocadas por equipo no parecen afectar tanto la posibilidad de que un equipo gane como las variables anteriores.
* Lo mismo se repite con los **minions totales asesinados** por equipo.

### Diferencias de Oro y Experiencia de un equipo

Para ver de una manera más explícita como contribuyen la **experiencia** y el **oro** a la victoria de un equipo podemos graficar la diferencia de estos valores en cada equipo:

In [ ]:
plt.figure(figsize=(20,20))
sns.set_style("whitegrid")

# Diferencia de experiencia y oro
plt.subplot(311)
sns.scatterplot(x='blueExperienceDiff', y='blueGoldDiff', hue='blueWins', data=data)
plt.title('Diferencia de ORO y EXPERIENCIA ')
plt.xlabel('Diferencia de Experiencia')
plt.ylabel('Diferencia de Oro')
plt.grid(True)

plt.show()

<a id="analisis-datos"></a>
## Análisis de Datos

### Oro y Experiencia

Siguiendo estas explicaciones, podemos deducir que:

> _A mayor cantidad de **kills** y **asistencias**_ 🡲 _Mayor cantidad de oro y experiencia para el equipo_ 

> _A mayor cantidad de **oro** y **experiencia**_ 🡲 _Mayor ventaja sobre el equipo enemigo_ 🡲 _Mayor probabilidad de ganar la partida_

Por lo tanto, podemos decir que el **oro** y la **experiencia** que posee un equipo respecto del otro, son variables **influyen enormemente** en las posibilidades de un equipo de ganar.  

### Dragones y Heraldos

Como se había descrito anteriormente, estos **monstruos de elite** otorgan **oro** y **experiencia** al equipo que los destruya, aunque también otorgan estadísticas permanentes que se van acumulando y potenciando a medida que el equipo destruya más, por lo que son un objetivo prioritario.

> _A mayor cantidad de **dragones** y **heraldos** asesinados_ 🡲 _Mejores estadísticas para el equipo_ 🡲 _Mayor probabilidad de ganar la partida_

### Torretas

Las torretas, asi como los inhibidores, son estructuras criticas que el equipo debe destruir si quiere ganar la partida. Ademas de que otorgan **oro** y **experiencia**.

### Wards

Como se observaban en los gráficos de las variables, la cantidad de **wards** colocadas no tienen un impacto muy grande dentro de lo que son las probabilidades de un equipo de ganar.

<a id="seleccion-variables"></a>
# Selección de Variables

Dadas las conclusiones obtenidas en el apartado de análisis de datos, luego de haber observado los gráficos, incluiremos las siguientes variables dentro del modelo, ya que son las que mas afectan las posibilidades de un equipo de ganar:
* **Diferencia de Kills.**
* **Diferencia de Asistencias.**
* **Diferencia de Heraldos.**
* **Diferencia de Dragones.**
* **Diferencia de Torres Destruidas.**
* **Diferencia de Oro.**
* **Diferencia de Experiencia.**

In [ ]:
# Generamos las columnas de datos que vamos a utilizar de acuerdo a lo propuesto anteriormente.
df['blueKillsDiff'] = df['blueKills'] - df['redKills']
df['blueAssistsDiff'] = df['blueAssists'] - df['redAssists']
df['blueHeraldsDiff'] = df['blueHeralds'] - df['redHeralds']
df['blueDragonsDiff'] = df['blueDragons'] - df['redDragons']
df['blueTowersDestroyedDiff'] = df['blueTowersDestroyed'] - df['redTowersDestroyed']

# Asignamos las columnas previamente generadas en una tabla nueva lista para ser usada por los clasificadores.
X = df[['blueKillsDiff', 'blueAssistsDiff', 'blueHeraldsDiff', 'blueDragonsDiff', 
        'blueTowersDestroyedDiff', 'blueGoldDiff', 'blueExperienceDiff']]

# Asignamos la variable objetivo.
y = df['blueWins']

# Imprimimos la tabla
X.head()

<a id="normalizacion-datos"></a>
# Normalización de los datos

Dado que existe una gran variación dentro del dataset en los valores que pueden tomar las diferentes variables, procederemos a normalizar los datos. Normalizar significa, en este caso, comprimir o extender los valores de la variable para que estén en un rango definido.

## Escalado Estándar (Standard Scaler)

En este caso, utilizaremos el **Escalado Estándar**, en donde a cada dato se le resta la **media** de la variable y se le divide por la **desviación típica**, segun la siguiente formula:

$$Xnormalized = \frac{X - Xmean}{Xstddev}$$

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix
from prettytable import PrettyTable

# Creamos la tabla que nos permitirá mostrar las métricas obtenidas.
metricas = PrettyTable()
metricas.field_names = ['Clasificador', 'Exactitud', 'Recall', 'Precisión']

# Guardaremos los resultados en un vector para ser mostrados en la conclusión del trabajo.
resultados = []

# Normalizamos los datos
X = StandardScaler().fit(X).transform(X)

# Asignamos los valores de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state = 4)

<a id="clasificadores-vanilla"></a>
# Clasificadores Vanilla

En primer lugar, utilizaremos los siguientes clasificadores usando los parámetros que vienen por defecto:
* Regresión Logística.
* K-Nearest Neighbours.
* Decision Tree.
* Random Forest.

## Regresión Logística

### Definición

Es un tipo de análisis de regresión utilizado para predecir el resultado de una variable categórica (una variable que puede adoptar un número limitado de categorías) en función de las variables independientes o predictoras. Es útil para modelar la probabilidad de un evento ocurriendo como función de otros factores. 

In [ ]:
from sklearn.linear_model import LogisticRegression

# Instanciamos el clasificador
LR = LogisticRegression()

# Hacemos fit a los datos y realizamos la predicción.
y_pred = LR.fit(X_train, y_train).predict(X_test)

# Métricas de evaluación
exactitud = accuracy_score(y_test,y_pred)
recall = recall_score(y_test,y_pred)
precision = precision_score(y_test,y_pred)
LR_confusion_matrix = confusion_matrix(y_test,y_pred)

# Anexamos los resultados para ser mostrados posteriormente
resultados.append(exactitud)

# Formateamos los datos para mostrarlos como % en la tabla.
exactitud = str(round(exactitud * 100, 2)) + " %"
recall = str(round(recall * 100, 2)) + " %"
precision = str(round(precision * 100, 2)) + " %"

metricas.add_row(['Regresión Logística', exactitud, recall, precision])

## K-Nearest Neighbours

### Definición

Es un algoritmo usado como método de clasificación de elementos, basado en un entrenamiento mediante ejemplos cercanos en el espacio que existe entre estos.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# Instanciamos el clasificador
KNN = KNeighborsClassifier()

# Hacemos fit a los datos y realizamos la predicción.
y_pred = KNN.fit(X_train, y_train).predict(X_test)

# Métricas de evaluación
exactitud = accuracy_score(y_test,y_pred)
recall = recall_score(y_test,y_pred)
precision = precision_score(y_test,y_pred)
KNN_confusion_matrix = confusion_matrix(y_test,y_pred)

# Anexamos los resultados para ser mostrados posteriormente
resultados.append(exactitud)

# Formateamos los datos para mostrarlos como % en la tabla.
exactitud = str(round(exactitud * 100, 2)) + " %"
recall = str(round(recall * 100, 2)) + " %"
precision = str(round(precision * 100, 2)) + " %"

metricas.add_row(['K-Nearest Neighbours', exactitud, recall, precision])

## Decision Tree

### Definición

Dado un conjunto de datos, se fabrican diagramas de reglas, que sirven para representar y categorizar una serie de condiciones que ocurren de forma sucesiva, para la resolución de un problema.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Instanciamos el clasificador
DT = DecisionTreeClassifier()

# Hacemos fit a los datos y realizamos la predicción.
y_pred = DT.fit(X_train, y_train).predict(X_test)

# Métricas de evaluación
exactitud = accuracy_score(y_test,y_pred)
recall = recall_score(y_test,y_pred)
precision = precision_score(y_test,y_pred)
DT_confusion_matrix = confusion_matrix(y_test,y_pred)

# Anexamos los resultados para ser mostrados posteriormente
resultados.append(exactitud)

# Formateamos los datos para mostrarlos como % en la tabla.
exactitud = str(round(exactitud * 100, 2)) + " %"
recall = str(round(recall * 100, 2)) + " %"
precision = str(round(precision * 100, 2)) + " %"

metricas.add_row(['Decision Tree', exactitud, recall, precision])

## Random Forest

### Definición

Es una combinación de árboles de decisión tal que cada árbol depende de los valores de un vector aleatorio probado independientemente y con la misma distribución para cada uno de estos.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Instanciamos el clasificador
RF = RandomForestClassifier()

# Hacemos fit a los datos y realizamos la predicción.
y_pred = RF.fit(X_train, y_train).predict(X_test)

# Métricas de evaluación
exactitud = accuracy_score(y_test,y_pred)
recall = recall_score(y_test,y_pred)
precision = precision_score(y_test,y_pred)
RF_confusion_matrix = confusion_matrix(y_test,y_pred)

# Anexamos los resultados para ser mostrados posteriormente
resultados.append(exactitud)

# Formateamos los datos para mostrarlos como % en la tabla.
exactitud = str(round(exactitud * 100, 2)) + " %"
recall = str(round(recall * 100, 2)) + " %"
precision = str(round(precision * 100, 2)) + " %"

metricas.add_row(['Random Forest', exactitud, recall, precision])

<a id="resultados-vanilla"></a>
## Resultados

<a id="metricas"></a>
### Métricas 

* **Accuracy (Exactitud):** Es el porcentaje total de elementos clasificados correctamente. Es la medida más directa de la calidad de los clasificadores. Es un valor entre 0 y 1. Cuanto más alto, mejor.

* **Recall (Tasa de True Positive):** Es el número de elementos identificados correctamente como positivos del total de positivos verdaderos.

* **Precision:** Es el número de elementos identificados correctamente como positivo de un total de elementos identificados como positivos.

* **Confusion Matrix (Matriz de Confusión):** Es una tabla que describe el rendimiento de un modelo supervisado de Machine Learning en los datos de prueba, donde se desconocen los verdaderos valores.

In [ ]:
# Imprimimos el título de la tabla.
print("Clasificadores Vanilla (Ordenados por Exactitud)")

# Ordenamos la tabla por la columna "Exactitud"
metricas.sortby = "Exactitud"

# Colocamos las filas en orden descendiente.
metricas.reversesort = True

# Imprimimos la tabla
print(metricas)

### Matrices de Confusión

In [ ]:
plt.figure(figsize=(20,10))

# Ploteamos la matriz de confusión de la 'Regresión Logistica'
plt.subplot(221)
sns.heatmap(LR_confusion_matrix, cmap="coolwarm", fmt=".0f",annot=True, linewidths=.5, annot_kws={"size": 16})
plt.xlabel("Prediccion")
plt.ylabel("Verdadero")
plt.title('Regresión Logistica')
plt.tight_layout(pad=1.5)

# Ploteamos la matriz de confusión de la 'K-Nearest Neighbours'
plt.subplot(222)
sns.heatmap(KNN_confusion_matrix, cmap="coolwarm", fmt=".0f",annot=True, linewidths=.5, annot_kws={"size": 16})
plt.xlabel("Prediccion")
plt.ylabel("Verdadero")
plt.title('K-Nearest Neighbours')
plt.tight_layout(pad=1.5)

# Ploteamos la matriz de confusión de la 'Decision Tree'
plt.subplot(223)
sns.heatmap(DT_confusion_matrix, cmap="coolwarm", fmt=".0f",annot=True, linewidths=.5, annot_kws={"size": 16})
plt.xlabel("Prediccion")
plt.ylabel("Verdadero")
plt.title('Decision Tree')
plt.tight_layout(pad=1.5)

# Ploteamos la matriz de confusión de la 'Random Forest'
plt.subplot(224)
sns.heatmap(RF_confusion_matrix, cmap="coolwarm", fmt=".0f",annot=True, linewidths=.5, annot_kws={"size": 16})
plt.xlabel("Prediccion")
plt.ylabel("Verdadero")
plt.title('Random Forest')
plt.tight_layout(pad=1.5)

plt.show()

<a id="grid-search"></a>
# Aplicando GridSearch + Cross Validation

## GridSearchCV

Es una clase disponible en **scikit-learn** que permite evaluar y seleccionar de forma sistemática los parámetros de un modelo. Indicándole un modelo y los parámetros a probar, puede evaluar el rendimiento del primero en función de los segundos mediante validación cruzada. 

## Validación Cruzada

Al hacer uso de esta tecnica, el conjunto de datos de entrenamiento se divide en grupos de igual tamaño. Una vez realizada la partición se procede a entrenar el modelo una vez por cada uno de los grupos. Utilizando todos los grupos menos el de la iteración para entrenar y este para validar los resultados. Como se aprecia en al siguiente imagen:

![](https://www.analyticslane.com/wp-content/uploads/2018/07/validacion_cruzada.jpeg.webp)

In [ ]:
# Importamos la clase de scikit-learn
from sklearn.model_selection import GridSearchCV

# Creamos la tabla que nos permitirá mostrar las métricas obtenidas.
metricas_grid_search = PrettyTable()
metricas_grid_search.field_names = ['Clasificador', 'Exactitud', 'Recall', 'Precisión']

# Guardaremos los resultados en un vector para ser mostrados en la conclusión del trabajo.
resultados_grid_search = []

<a id="clasificadores-grid"></a>
## Clasificadores

### Regresión Logística

In [ ]:
# Colocamos los valores de parámetros que queremos que GridSearchCV pruebe por nosotros
grid_values = {'penalty': ['l1', 'l2'],
               'C':[.001,.009,0.01,.09,1,2,3,4,5,7,10,25],
               'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
               'fit_intercept' : [True, False]}

# Instanciamos la clase con los parámetros previamente asignados
grid_clf_acc = GridSearchCV(LogisticRegression(), param_grid = grid_values, scoring = 'accuracy', verbose=False, n_jobs=-1)

# Seleccionamos la tabla entera, ya que el método se encargará de realizar la técnica de Cross-Validation
grid_clf_acc.fit(X, y)

# Imprimimos los mejores parámetros seleccionados por GridSearchCV
print("Parámetros elegidos: " + str(grid_clf_acc.best_params_) + "\n")

# Predecimos los valores
y_pred_acc = grid_clf_acc.predict(X)

# Métricas de evaluación
exactitud = accuracy_score(y,y_pred_acc)
recall = recall_score(y,y_pred_acc)
precision = precision_score(y,y_pred_acc)
LR_confusion_matrix = confusion_matrix(y,y_pred_acc)

# Anexamos los resultados para ser mostrados posteriormente
resultados_grid_search.append(exactitud)

# Formateamos los datos para mostrarlos como % en la tabla.
exactitud = str(round(exactitud * 100, 2)) + " %"
recall = str(round(recall * 100, 2)) + " %"
precision = str(round(precision * 100, 2)) + " %"

metricas_grid_search.add_row(['Regresión Logística', exactitud, recall, precision])

### K-Nearest Neighbours

In [ ]:
# Colocamos los valores de parámetros que queremos que GridSearchCV pruebe por nosotros
grid_values = {"n_neighbors": [3, 4, 5, 6, 7],
                 "weights": ["uniform","distance"],
                 "metric":["euclidean","manhattan"]}

# Instanciamos la clase con los parámetros previamente asignados
grid_clf_acc = GridSearchCV(KNeighborsClassifier(), param_grid = grid_values, scoring = 'accuracy', verbose=False, n_jobs=-1)

# Seleccionamos la tabla entera, ya que el método se encargará de realizar la técnica de Cross-Validation
grid_clf_acc.fit(X, y)

# Imprimimos los mejores parámetros seleccionados por GridSearchCV
print("Parámetros elegidos: " + str(grid_clf_acc.best_params_) + "\n")

# Predecimos los valores
y_pred_acc = grid_clf_acc.predict(X)

# Métricas de evaluación
exactitud = accuracy_score(y,y_pred_acc)
recall = recall_score(y,y_pred_acc)
precision = precision_score(y,y_pred_acc)
KNN_confusion_matrix = confusion_matrix(y,y_pred_acc)

# Anexamos los resultados para ser mostrados posteriormente
resultados_grid_search.append(exactitud)

# Formateamos los datos para mostrarlos como % en la tabla.
exactitud = str(round(exactitud * 100, 2)) + " %"
recall = str(round(recall * 100, 2)) + " %"
precision = str(round(precision * 100, 2)) + " %"

metricas_grid_search.add_row(['K-Nearest Neighbours', exactitud, recall, precision])

### Decision Tree

In [ ]:
# Colocamos los valores de parámetros que queremos que GridSearchCV pruebe por nosotros
grid_values = {'max_depth': np.arange(1, 21),
               'min_samples_leaf': [1, 5, 10, 20, 50, 100]}

# Instanciamos la clase con los parámetros previamente asignados
grid_clf_acc = GridSearchCV(DecisionTreeClassifier(), param_grid = grid_values, scoring = 'accuracy', verbose=False, n_jobs=-1)

# Seleccionamos la tabla entera, ya que el método se encargará de realizar la técnica de Cross-Validation
grid_clf_acc.fit(X, y)

# Imprimimos los mejores parámetros seleccionados por GridSearchCV
print("Parámetros elegidos: " + str(grid_clf_acc.best_params_) + "\n")

# Predecimos los valores
y_pred_acc = grid_clf_acc.predict(X)

# Métricas de evaluación
exactitud = accuracy_score(y,y_pred_acc)
recall = recall_score(y,y_pred_acc)
precision = precision_score(y,y_pred_acc)
DT_confusion_matrix = confusion_matrix(y,y_pred_acc)

# Anexamos los resultados para ser mostrados posteriormente
resultados_grid_search.append(exactitud)

# Formateamos los datos para mostrarlos como % en la tabla.
exactitud = str(round(exactitud * 100, 2)) + " %"
recall = str(round(recall * 100, 2)) + " %"
precision = str(round(precision * 100, 2)) + " %"

metricas_grid_search.add_row(['Decision Tree', exactitud, recall, precision])

### Random Forest

In [ ]:
# Colocamos los valores de parámetros que queremos que GridSearchCV pruebe por nosotros
grid_values = {'max_features': ['auto', 'sqrt', 'log2'],
                'max_depth' : [4, 5, 6, 7, 8],
                'criterion' :['gini', 'entropy']}

# Instanciamos la clase con los parámetros previamente asignados
grid_clf_acc = GridSearchCV(RandomForestClassifier(), param_grid = grid_values, scoring = 'accuracy', verbose=False, n_jobs=-1)

# Seleccionamos la tabla entera, ya que el método se encargará de realizar la técnica de Cross-Validation
grid_clf_acc.fit(X, y)

# Imprimimos los mejores parámetros seleccionados por GridSearchCV
print("Parámetros elegidos: " + str(grid_clf_acc.best_params_) + "\n")

# Predecimos los valores
y_pred_acc = grid_clf_acc.predict(X)

# Métricas de evaluación
exactitud = accuracy_score(y,y_pred_acc)
recall = recall_score(y,y_pred_acc)
precision = precision_score(y,y_pred_acc)
RF_confusion_matrix = confusion_matrix(y,y_pred_acc)

# Anexamos los resultados para ser mostrados posteriormente
resultados_grid_search.append(exactitud)

# Formateamos los datos para mostrarlos como % en la tabla.
exactitud = str(round(exactitud * 100, 2)) + " %"
recall = str(round(recall * 100, 2)) + " %"
precision = str(round(precision * 100, 2)) + " %"

metricas_grid_search.add_row(['Random Forest', exactitud, recall, precision])

<a id="resultados-grid"></a>
## Resultados

### Métricas

In [ ]:
print("Clasificadores con GridSearchCV (Ordenados por Exactitud)")
metricas_grid_search.sortby = "Exactitud"
metricas_grid_search.reversesort = True
print(metricas_grid_search)

### Matrices de Confusión

In [ ]:
plt.figure(figsize=(20,10))

plt.subplot(221)
sns.heatmap(LR_confusion_matrix, cmap="coolwarm", fmt=".0f",annot=True, linewidths=.5, annot_kws={"size": 16})
plt.xlabel("Prediccion")
plt.ylabel("Verdadero")
plt.title('Regresión Logistica')
plt.tight_layout(pad=1.5)

plt.subplot(222)
sns.heatmap(KNN_confusion_matrix, cmap="coolwarm", fmt=".0f",annot=True, linewidths=.5, annot_kws={"size": 16})
plt.xlabel("Prediccion")
plt.ylabel("Verdadero")
plt.title('K-Nearest Neighbours')
plt.tight_layout(pad=1.5)

plt.subplot(223)
sns.heatmap(DT_confusion_matrix, cmap="coolwarm", fmt=".0f",annot=True, linewidths=.5, annot_kws={"size": 16})
plt.xlabel("Prediccion")
plt.ylabel("Verdadero")
plt.title('Decision Tree')
plt.tight_layout(pad=1.5)

plt.subplot(224)
sns.heatmap(RF_confusion_matrix, cmap="coolwarm", fmt=".0f",annot=True, linewidths=.5, annot_kws={"size": 16})
plt.xlabel("Prediccion")
plt.ylabel("Verdadero")
plt.title('Random Forest')
plt.tight_layout(pad=1.5)

plt.show()

<a id="pca"></a>
# Aplicando PCA + GridSearch + Cross Validation

## Análisis de Componentes Principales (PCA)

Es una técnica de selección de características, que utiliza una transformación ortogonal para convertir un conjunto de observaciones de variables, posiblemente correlacionadas, en un conjunto más reducido de variables que ya no guardan correlación y que se conocen como **componentes principales**. 

Al realizar este análisis, intentamos buscar cuantos parámetros mínimos son necesarios para explicar una cantidad significativa de variación en los datos, es decir, valorar cuánta información nos podemos permitir descartar al eliminar ciertos parámetros y, de esta manera, obtener un modelo más rápido y eficiente. 

En este caso, podemos deconstruir un conjunto de datos en:
* **Autovectores:** es una dirección.
* **Autovalores:**: es un número que representa el valor de la varianza en esa dirección.

Por lo tanto, el componente principal será el autovector con mayor autovalor. En un conjunto de datos hay tantas parejas autovector/autovalor como dimensiones. 

### Aplicado a nuestro caso

En nuestro caso en específico, tenemos 7 dimensiones, dadas por cada uno de los features que hemos elegido previamente. Por lo que vamos a proceder a calcular los **autovalores** y **autovectores** de los datos, ver cuanta covarianza explica cada uno y, de esta manera, buscar reducir la dimensionalidad del conjunto de datos para obtener un modelo más rápido, eficiente y con la mínima perdida de información posible.

In [ ]:
# Calculamos la matriz de covarianza
matriz_covarianza = np.cov(X.T)

# Calculamos los autovalores y autovectores de la matriz
auto_valores, auto_vectores = np.linalg.eig(matriz_covarianza)
                                  
# A partir de los autovalores, calculamos la varianza explicada individual y la acumulada
total = sum(auto_valores)
varianza_explicada = [(i / total) * 100 for i in sorted(auto_valores, reverse=True)]
varianza_explicada_acumulada = np.cumsum(varianza_explicada)

# Graficamos la varianza explicada por cada autovalor, y la acumulada
plt.figure(figsize=(20,10))

plt.bar(range(7), varianza_explicada, alpha=0.5, align='center',label='Varianza individual explicada', color='b')
plt.step(range(7), varianza_explicada_acumulada, where='mid', linestyle='-', label='Varianza explicada acumulada', color='r')
plt.ylabel('Varianza Explicada')
plt.xlabel('Componentes Principales')
plt.legend(loc='best')
plt.tight_layout()

plt.show()

### Evaluación del Gráfico

Como podemos advertir en el grafico anterior, las primeras 4 componentes explican, por lo menos, el 90% de la varianza de los datos. Por lo que procederemos a reducir la dimensionalidad de los datos utilizando estas 4 componentes.

<a id="implementar-pca"></a>
## Implementando PCA de Scikit-Learn

In [ ]:
# Importamos la clase de scikit-learn
from sklearn.decomposition import PCA

# Creamos la tabla que nos permitirá mostrar las métricas obtenidas.
metricas_pca = PrettyTable()
metricas_pca.field_names = ['Clasificador', 'Exactitud', 'Recall', 'Precisión']

# Guardaremos los resultados en un vector para ser mostrados en la conclusión del trabajo.
resultados_pca = []

# Instanciamos una clase de PCA indicandole que utilizaremos 4 componentes principales
pca = PCA(n_components = 4)
X_PCA = pca.fit(X).transform(X)

<a id="clasificadores-pca"></a>
## Clasificadores

### Regresión Logística

In [ ]:
# Colocamos los valores de parámetros que queremos que GridSearchCV pruebe por nosotros
grid_values = {'penalty': ['l1', 'l2'],
               'C':[.001,.009,0.01,.09,1,2,3,4,5,7,10,25],
               'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
               'fit_intercept' : [True, False]}

# Instanciamos la clase con los parámetros previamente asignados
grid_clf_acc = GridSearchCV(LogisticRegression(), param_grid = grid_values, scoring = 'accuracy', verbose=False, n_jobs=-1)

# Seleccionamos la tabla entera, ya que el método se encargará de realizar la técnica de Cross-Validation
grid_clf_acc.fit(X_PCA, y)

# Imprimimos los mejores parámetros seleccionados por GridSearchCV
print("Parámetros elegidos: " + str(grid_clf_acc.best_params_) + "\n")

# Predecimos los valores
y_pred_acc = grid_clf_acc.predict(X_PCA)

# Métricas de evaluación
exactitud = accuracy_score(y,y_pred_acc)
recall = recall_score(y,y_pred_acc)
precision = precision_score(y,y_pred_acc)
LR_confusion_matrix = confusion_matrix(y,y_pred_acc)

# Anexamos los resultados para ser mostrados posteriormente
resultados_pca.append(exactitud)

# Formateamos los datos para mostrarlos como % en la tabla.
exactitud = str(round(exactitud * 100, 2)) + " %"
recall = str(round(recall * 100, 2)) + " %"
precision = str(round(precision * 100, 2)) + " %"

metricas_pca.add_row(['Regresión Logística', exactitud, recall, precision])

### K-Nearest Neighbours

In [ ]:
# Colocamos los valores de parámetros que queremos que GridSearchCV pruebe por nosotros
grid_values = {"n_neighbors": [3, 4, 5, 6, 7],
                 "weights": ["uniform","distance"],
                 "metric":["euclidean","manhattan"]}

# Instanciamos la clase con los parámetros previamente asignados
grid_clf_acc = GridSearchCV(KNeighborsClassifier(), param_grid = grid_values, scoring = 'accuracy', verbose=False, n_jobs=-1)

# Seleccionamos la tabla entera, ya que el método se encargará de realizar la técnica de Cross-Validation
grid_clf_acc.fit(X_PCA, y)

# Imprimimos los mejores parámetros seleccionados por GridSearchCV
print("Parámetros elegidos: " + str(grid_clf_acc.best_params_) + "\n")

# Predecimos los valores
y_pred_acc = grid_clf_acc.predict(X_PCA)

# Métricas de evaluación
exactitud = accuracy_score(y,y_pred_acc)
recall = recall_score(y,y_pred_acc)
precision = precision_score(y,y_pred_acc)
KNN_confusion_matrix = confusion_matrix(y,y_pred_acc)

# Anexamos los resultados para ser mostrados posteriormente
resultados_pca.append(exactitud)

# Formateamos los datos para mostrarlos como % en la tabla.
exactitud = str(round(exactitud * 100, 2)) + " %"
recall = str(round(recall * 100, 2)) + " %"
precision = str(round(precision * 100, 2)) + " %"

metricas_pca.add_row(['K-Nearest Neighbours', exactitud, recall, precision])

### Decision Tree

In [ ]:
# Colocamos los valores de parámetros que queremos que GridSearchCV pruebe por nosotros
grid_values = {'max_depth': np.arange(1, 21),
               'min_samples_leaf': [1, 5, 10, 20, 50, 100]}

# Instanciamos la clase con los parámetros previamente asignados
grid_clf_acc = GridSearchCV(DecisionTreeClassifier(), param_grid = grid_values, scoring = 'accuracy', verbose=False, n_jobs=-1)

# Seleccionamos la tabla entera, ya que el método se encargará de realizar la técnica de Cross-Validation
grid_clf_acc.fit(X_PCA, y)

# Imprimimos los mejores parámetros seleccionados por GridSearchCV
print("Parámetros elegidos: " + str(grid_clf_acc.best_params_) + "\n")

# Predecimos los valores
y_pred_acc = grid_clf_acc.predict(X_PCA)

# Métricas de evaluación
exactitud = accuracy_score(y,y_pred_acc)
recall = recall_score(y,y_pred_acc)
precision = precision_score(y,y_pred_acc)
DT_confusion_matrix = confusion_matrix(y,y_pred_acc)

# Anexamos los resultados para ser mostrados posteriormente
resultados_pca.append(exactitud)

# Formateamos los datos para mostrarlos como % en la tabla.
exactitud = str(round(exactitud * 100, 2)) + " %"
recall = str(round(recall * 100, 2)) + " %"
precision = str(round(precision * 100, 2)) + " %"

metricas_pca.add_row(['Decision Tree', exactitud, recall, precision])

### Random Forest

In [ ]:
# Colocamos los valores de parámetros que queremos que GridSearchCV pruebe por nosotros
grid_values = {'max_features': ['auto', 'sqrt', 'log2'],
                'max_depth' : [4, 5, 6, 7, 8],
                'criterion' :['gini', 'entropy']}

# Instanciamos la clase con los parámetros previamente asignados
grid_clf_acc = GridSearchCV(RandomForestClassifier(), param_grid = grid_values, scoring = 'accuracy', verbose=False, n_jobs=-1)

# Seleccionamos la tabla entera, ya que el método se encargará de realizar la técnica de Cross-Validation
grid_clf_acc.fit(X_PCA, y)

# Imprimimos los mejores parámetros seleccionados por GridSearchCV
print("Parámetros elegidos: " + str(grid_clf_acc.best_params_) + "\n")

# Predecimos los valores
y_pred_acc = grid_clf_acc.predict(X_PCA)

# Métricas de evaluación
exactitud = accuracy_score(y,y_pred_acc)
recall = recall_score(y,y_pred_acc)
precision = precision_score(y,y_pred_acc)
RF_confusion_matrix = confusion_matrix(y,y_pred_acc)

# Anexamos los resultados para ser mostrados posteriormente
resultados_pca.append(exactitud)

# Formateamos los datos para mostrarlos como % en la tabla.
exactitud = str(round(exactitud * 100, 2)) + " %"
recall = str(round(recall * 100, 2)) + " %"
precision = str(round(precision * 100, 2)) + " %"

metricas_pca.add_row(['Random Forest', exactitud, recall, precision])

<a id="resultados-pca"></a>
## Resultados

### Métricas 

In [ ]:
print("Clasificadores con PCA + GridSearch + Cross Validation (Ordenados por Exactitud)")
metricas_pca.sortby = "Exactitud"
metricas_pca.reversesort = True
print(metricas_pca)

### Matrices de Confusión

In [ ]:
plt.figure(figsize=(20,10))

plt.subplot(221)
sns.heatmap(LR_confusion_matrix, cmap="coolwarm", fmt=".0f",annot=True, linewidths=.5, annot_kws={"size": 16})
plt.xlabel("Prediccion")
plt.ylabel("Verdadero")
plt.title('Regresión Logistica')
plt.tight_layout(pad=1.5)

plt.subplot(222)
sns.heatmap(KNN_confusion_matrix, cmap="coolwarm", fmt=".0f",annot=True, linewidths=.5, annot_kws={"size": 16})
plt.xlabel("Prediccion")
plt.ylabel("Verdadero")
plt.title('K-Nearest Neighbours')
plt.tight_layout(pad=1.5)

plt.subplot(223)
sns.heatmap(DT_confusion_matrix, cmap="coolwarm", fmt=".0f",annot=True, linewidths=.5, annot_kws={"size": 16})
plt.xlabel("Prediccion")
plt.ylabel("Verdadero")
plt.title('Decision Tree')
plt.tight_layout(pad=1.5)

plt.subplot(224)
sns.heatmap(RF_confusion_matrix, cmap="coolwarm", fmt=".0f",annot=True, linewidths=.5, annot_kws={"size": 16})
plt.xlabel("Prediccion")
plt.ylabel("Verdadero")
plt.title('Random Forest')
plt.tight_layout(pad=1.5)

plt.show()

<a id="conclusion"></a>
# Conclusión

In [ ]:
plt.figure(figsize=(20,15))
sns.set_style("whitegrid")

modelos = ["Regresión Logística", "K-Nearest Neighbours", "Decision Tree", "Random Forest"]
grafico_resultados = pd.DataFrame({"Puntuación": resultados, "Modelos": modelos})
grafico_resultados_grid_search = pd.DataFrame({"Puntuación": resultados_grid_search, "Modelos": modelos})
grafico_resultados_pca = pd.DataFrame({"Puntuación": resultados_pca, "Modelos": modelos})

plt.subplot(311)
sns.barplot("Puntuación", "Modelos", data = grafico_resultados)
plt.ylabel("")
plt.title('Clasificadores Vanilla')
plt.tight_layout(pad=2)

plt.subplot(312)
sns.barplot("Puntuación", "Modelos", data = grafico_resultados_grid_search)
plt.ylabel("")
plt.title('Clasificadores con GridSearch + Cross Validation')
plt.tight_layout(pad=2)

plt.subplot(313)
sns.barplot("Puntuación", "Modelos", data = grafico_resultados_pca)
plt.title('Clasificadores con PCA + GridSearch + Cross Validation')
plt.ylabel("")

plt.show()

## Observación de los Gráficos

Gracias a los graficos previos, podemos observar que:
* La **Regresión Logística** tiene una mayor exactitud si lo comparamos al resto de clasificadores utilizando los parámetros por defecto.
* Cuando aplicamos **Grid Search** y **PCA**, podemos observar que el algoritmo de **KNN** tiende a precedir mejor los resultados que el resto de los métodos.
* Tambien podemos notar que los demás metodos, como son **Decision Tree** y **Random Forest**, que antes tenian un porcentaje de exactitud menor, han aumentado dicha metrica y se han colocado por encima del algoritmo que antes se encontraba liderando, es decir, la **Regresión Logística.**

<a id="tabla-metricas"></a>
## Tabla de Métricas

In [ ]:
metricas.sortby = "Clasificador"
metricas_grid_search.sortby = "Clasificador"
metricas_pca.sortby = "Clasificador"

print("Clasificadores Vanilla")
print(metricas)
print("")
print("Clasificadores con GridSearch + Cross Validation")
print(metricas_grid_search)
print("")
print("Clasificadores con PCA + GridSearch + Cross Validation")
print(metricas_pca)

<a id="bibliografia"></a>
# Bibliografía

## Uso de Herramientas y Clasificadores

* [matplotlib](https://matplotlib.org/3.2.1/api/index.html)
* [numpy](https://numpy.org/doc/1.18/reference/index.html)
* [pandas](https://pandas.pydata.org/docs/reference/frame.html)
* [seaborn](https://seaborn.pydata.org/api.html)
* [scikit-learn](https://scikit-learn.org/stable/modules/classes.html)

## Conceptos Teoricos
* [Concepto y uso de GridSearchCV](https://www.analyticslane.com/2018/07/02/gridsearchcv/)
* [Matriz de Correlación](https://support.minitab.com/es-mx/minitab/18/help-and-how-to/modeling-statistics/multivariate/how-to/item-analysis/interpret-the-results/all-statistics-and-graphs/)
* [Matriz de Confusión](https://empresas.blogthinkbig.com/ml-a-tu-alcance-matriz-confusion/)
* [GitHub de la cátedra](https://github.com/inteligenciafrvm/inteligenciafrvm)
* [Normalización de los datos](https://empresas.blogthinkbig.com/precauciones-la-hora-de-normalizar/)
* [Métricas de Clasificación](https://sitiobigdata.com/2019/01/19/machine-learning-metrica-clasificacion-parte-3/#)
* [Análisis de Componentes Principales (PCA)](https://empresas.blogthinkbig.com/python-para-todos-que-es-el-pca/)